# KNN Regression

KNN in our problem might not be the best choice - variables highly skewed! But we will do our best!

We will apply following approach in case of KNN:
 * we will perform feature engineering - standardization
 * then we will find "good enough" parameters (in CV) to proceed feature selection procedure and after that we will have groups of feature candidates
 * then we will tune hyperparameters for each group of variables (in CV) - we will obtain couple of models
 * then we will compare all models based on so called "proper CV" and we will fit and picke the winner!

We are aware of potential data leakage in case of usage KFold CV without time-series problem handling (in point 2 and 3). 

*To be honest in this problem it is not a big deal - based on our experience and we treat it like a feature!*

During the last step of our procedure we will verify previous analysis based on "proper CV" which handle time-series properties!!! So during hyperparameters tuning we will use different CV (we fight against data leakage).

### Dependencies loading

In [107]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import RFECV
from sklearn.inspection import permutation_importance
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from ReliefF import ReliefF
import pickle

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 150)

# np.random.seed(1916) #uncomment if you want your code to be reproducible; for the purposes of our activity, let's add some randomness to the results

### Data loading

In [108]:
preprocessed_output_data_path = "../data/output"

df = pd.read_csv(f"{preprocessed_output_data_path}/train_fe.csv", index_col=0)

fr = pd.read_excel(f"{preprocessed_output_data_path}/feature_ranking.xlsx", index_col=0)

### Feature engineering for KNN model

We have to standardize our variables. We will use range standardization (Min Max Scaler) because we have got dummies! We gave every variable a chance to have the same impact on the model.

In [109]:
print(df.columns.tolist())

['Ticker', 'Nazwa2', 'rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'etr', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'capex2_scaled', 'y_v2x_polyarchy', 'y_e_p_polity', 'y_BR_Democracy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'ta_log', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_c

In [110]:
columns = [
    "rok",
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "etr",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "capex2_scaled",
    "y_v2x_polyarchy",
    "y_e_p_polity",
    "y_BR_Democracy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "sektor_consumer discretionary",
    "sektor_consumer staples",
    "sektor_energy",
    "sektor_health care",
    "sektor_industrials",
    "sektor_materials",
    "sektor_real estate",
    "sektor_technology",
    "sektor_utilities",
    "gielda_2",
    "gielda_3",
    "gielda_4",
    "gielda_5",
    "ta_log",
    "txt_cat_(-63.011, -34.811]",
    "txt_cat_(-34.811, 0.488]",
    "txt_cat_(0.488, 24.415]",
    "txt_cat_(24.415, 25.05]",
    "txt_cat_(25.05, 308.55]",
    "txt_cat_(308.55, 327.531]",
    "txt_cat_(327.531, inf]",
    "pi_cat_(-8975.0, -1.523]",
    "pi_cat_(-1.523, 157.119]",
    "pi_cat_(157.119, 465.9]",
    "pi_cat_(465.9, 7875.5]",
    "pi_cat_(7875.5, 8108.5]",
    "pi_cat_(8108.5, inf]",
    "str_cat_(0.0875, 0.192]",
    "str_cat_(0.192, 0.28]",
    "str_cat_(0.28, inf]",
    "xrd_exists",
    "ni_profit",
    "ni_profit_20000",
    "ppent_sqrt",
    "intant_sqrt",
    "dlc_cat_(42.262, 176.129]",
    "dlc_cat_(176.129, 200.9]",
    "dlc_cat_(200.9, inf]",
    "dltt_cat_(39.38, 327.85]",
    "dltt_cat_(327.85, 876.617]",
    "dltt_cat_(876.617, inf]",
    "capex_cat_(7.447, 79.55]",
    "capex_cat_(79.55, 5451.0]",
    "capex_cat_(5451.0, inf]",
    "revenue_cat_(0.174, 1248.817]",
    "revenue_cat_(1248.817, 4233.587]",
    "revenue_cat_(4233.587, inf]",
    "cce_cat_(5.619, 63.321]",
    "cce_cat_(63.321, inf]",
    "adv_cat_(0.3, 874.5]",
    "adv_cat_(874.5, inf]",
    "diff_positive",
    "roa_clip",
    "lev_sqrt",
    "intan_pow2",
    "rd_sqrt",
    "ppe_clip",
    "cash_holdings_sqrt",
    "adv_expenditure_positive",
    "diff_dta",
    "cfc_dta",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]

In [111]:
standardization = list()
not_standardization = list()
for i in columns:
    if df[i].nunique() > 2:
        standardization.append(i)
    else:
        not_standardization.append(i)

In [112]:
print(standardization)

['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'etr', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'capex2_scaled', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'ta_log', 'ppent_sqrt', 'intant_sqrt', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'diff_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']


In [113]:
print(not_standardization)

['cfc', 'dta', 'y_e_p_polity', 'y_BR_Democracy', 'sektor_consumer discretionary', 'sektor_consumer staples', 'sektor_energy', 'sektor_health care', 'sektor_industrials', 'sektor_materials', 'sektor_real estate', 'sektor_technology', 'sektor_utilities', 'gielda_2', 'gielda_3', 'gielda_4', 'gielda_5', 'txt_cat_(-63.011, -34.811]', 'txt_cat_(-34.811, 0.488]', 'txt_cat_(0.488, 24.415]', 'txt_cat_(24.415, 25.05]', 'txt_cat_(25.05, 308.55]', 'txt_cat_(308.55, 327.531]', 'txt_cat_(327.531, inf]', 'pi_cat_(-8975.0, -1.523]', 'pi_cat_(-1.523, 157.119]', 'pi_cat_(157.119, 465.9]', 'pi_cat_(465.9, 7875.5]', 'pi_cat_(7875.5, 8108.5]', 'pi_cat_(8108.5, inf]', 'str_cat_(0.0875, 0.192]', 'str_cat_(0.192, 0.28]', 'str_cat_(0.28, inf]', 'xrd_exists', 'ni_profit', 'ni_profit_20000', 'dlc_cat_(42.262, 176.129]', 'dlc_cat_(176.129, 200.9]', 'dlc_cat_(200.9, inf]', 'dltt_cat_(39.38, 327.85]', 'dltt_cat_(327.85, 876.617]', 'dltt_cat_(876.617, inf]', 'capex_cat_(7.447, 79.55]', 'capex_cat_(79.55, 5451.0]', '

In [114]:
standardization.remove("etr")

In [115]:
standardization.append("y_e_p_polity")

In [116]:
print(standardization)

['rok', 'ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'capex2_scaled', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'ta_log', 'ppent_sqrt', 'intant_sqrt', 'roa_clip', 'lev_sqrt', 'intan_pow2', 'rd_sqrt', 'ppe_clip', 'cash_holdings_sqrt', 'diff_dta', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past', 'y_e_p_polity']


In [117]:
scaler = MinMaxScaler()
scaler.fit(df[standardization])
df[standardization] = scaler.transform(df[standardization])

In [118]:
pickle.dump(scaler, open("../models/minmaxscaler.sav", "wb"))

In [119]:
# df[standardization]= scaler.fit_transform(df[standardization])

Double check. Everything seems to be good :-) 

In [120]:
df[columns].describe().T["min"].unique()

array([0., 1.])

In [121]:
df[columns].describe().T["max"].unique()

array([1., 1., 1.])

### Searching for "good enough" model to feature selection

Let's use top 10 variables proposed by Mutual Information!

In [122]:
var = fr.mi_score.sort_values(ascending=False).index.tolist()[0:10]

In [123]:
print(var)

['etr_y_past', 'etr_y_ma', 'txt', 'diff', 'ni', 'pi', 'intant', 'intant_sqrt', 'ta', 'revenue']


In [124]:
df.shape[0] ** (0.5)

63.190189111918315

In [125]:
param = {
    "n_neighbors": [5, 7, 10, 12, 15, 25, 40, 50, 100],
    "weights": ["uniform", "distance"],
    "metric": ["minkowski", "manhattan", "chebyshev"],
    "p": [1, 2],
}

In [126]:
mse = make_scorer(mean_squared_error, greater_is_better=False)

In [127]:
model = KNeighborsRegressor()
grid_CV = GridSearchCV(
    model, param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1
)
grid_CV.fit(df.loc[:, var].values, df.loc[:, "etr"].values.ravel())

GridSearchCV(cv=5, estimator=KNeighborsRegressor(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'manhattan', 'chebyshev'],
                         'n_neighbors': [5, 7, 10, 12, 15, 25, 40, 50, 100],
                         'p': [1, 2], 'weights': ['uniform', 'distance']},
             return_train_score=True,
             scoring=make_scorer(mean_squared_error, greater_is_better=False))

In [128]:
grid_CV.best_params_

{'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'uniform'}

In [129]:
grid_CV.cv_results_

{'mean_fit_time': array([0.00079145, 0.00117559, 0.00211382, 0.00173306, 0.00185461,
        0.00179081, 0.00144186, 0.0021678 , 0.0016046 , 0.00155983,
        0.00122328, 0.00068159, 0.00092759, 0.00152526, 0.00128031,
        0.00146418, 0.00145063, 0.0010468 , 0.00094976, 0.0014924 ,
        0.00098848, 0.00067887, 0.00092816, 0.00068731, 0.00118065,
        0.00093632, 0.00089221, 0.00066123, 0.00106959, 0.00067391,
        0.00067306, 0.00089197, 0.00164022, 0.00087276, 0.0006938 ,
        0.0009758 , 0.00172186, 0.00088525, 0.00090733, 0.00145345,
        0.00304723, 0.00091658, 0.00155573, 0.00151649, 0.00109611,
        0.00096154, 0.00105014, 0.0025744 , 0.0020596 , 0.00096135,
        0.00085711, 0.00088716, 0.00179014, 0.00066762, 0.00181532,
        0.00166287, 0.00132513, 0.00110188, 0.00104022, 0.00087256,
        0.00158472, 0.00153637, 0.0008779 , 0.00067863, 0.00130787,
        0.00090456, 0.00097289, 0.00092525, 0.00073557, 0.0024652 ,
        0.00119743, 0.00069785,

Right now (temporary) we will this hyperparameters as the best one:

In [130]:
grid_CV.best_params_

{'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'uniform'}

### Feature selection for KNN

We would like to base our feature selection on:
 * feature ranking
 * forward elimination
 * relief - special feature selection model for KNN found during Michał Woźniak & Michał Dyczko undergrad. research

#### Feature ranking

In [131]:
fr.sort_values("mi_score", ascending=False, inplace=True)

In [132]:
fr.head()

,mi_score,sign_fscore,sign_fscore_0_1,corr,EN_coef,boruta_rank
etr_y_past,1.009402,1.304040e-84,1,0.520405,NaN,1
etr_y_ma,0.825650,2.473770e-125,1,0.526871,NaN,1
txt,0.633067,5.246456e-13,1,0.368732,1.466269e-05,1
diff,0.632640,2.257712e-02,1,-0.291716,NaN,1
ni,0.613297,1.747230e-09,1,0.263458,-3.442000e-07,7


In [133]:
br_features = fr[fr.boruta_rank.isin([1, 2, 3])].index.tolist()

In [134]:
mi_features = fr.iloc[0:20].index.tolist()

In [135]:
mi_features_25 = fr.iloc[0:25].index.tolist()

In [136]:
mi_features_35 = fr.iloc[0:35].index.tolist()

In [137]:
mi_features_50 = fr.iloc[0:50].index.tolist()

In [138]:
fr["corr_abs"] = np.abs(fr["corr"])
fr.sort_values("corr_abs", ascending=False, inplace=True)
corr_features = fr.iloc[0:20].index.tolist()

We will use our intuition and create two additional benchmark sets of variables:

In [139]:
benchmark = [
    "rok",
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "capex2_scaled",
    "y_v2x_polyarchy",
    "y_e_p_polity",
    "y_BR_Democracy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "sektor_consumer discretionary",
    "sektor_consumer staples",
    "sektor_energy",
    "sektor_health care",
    "sektor_industrials",
    "sektor_materials",
    "sektor_real estate",
    "sektor_technology",
    "sektor_utilities",
    "gielda_2",
    "gielda_3",
    "gielda_4",
    "gielda_5",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]

In [140]:
benchmark2 = [
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "y_v2x_polyarchy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]

#### Forward elimination

In [141]:
forward_elimination = [
    "rok",
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "capex2_scaled",
    "y_v2x_polyarchy",
    "y_e_p_polity",
    "y_BR_Democracy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "sektor_consumer discretionary",
    "sektor_consumer staples",
    "sektor_energy",
    "sektor_health care",
    "sektor_industrials",
    "sektor_materials",
    "sektor_real estate",
    "sektor_technology",
    "sektor_utilities",
    "gielda_2",
    "gielda_3",
    "gielda_4",
    "gielda_5",
    "ta_log",
    "txt_cat_(-63.011, -34.811]",
    "txt_cat_(-34.811, 0.488]",
    "txt_cat_(0.488, 24.415]",
    "txt_cat_(24.415, 25.05]",
    "txt_cat_(25.05, 308.55]",
    "txt_cat_(308.55, 327.531]",
    "txt_cat_(327.531, inf]",
    "pi_cat_(-8975.0, -1.523]",
    "pi_cat_(-1.523, 157.119]",
    "pi_cat_(157.119, 465.9]",
    "pi_cat_(465.9, 7875.5]",
    "pi_cat_(7875.5, 8108.5]",
    "pi_cat_(8108.5, inf]",
    "str_cat_(0.0875, 0.192]",
    "str_cat_(0.192, 0.28]",
    "str_cat_(0.28, inf]",
    "xrd_exists",
    "ni_profit",
    "ni_profit_20000",
    "ppent_sqrt",
    "intant_sqrt",
    "dlc_cat_(42.262, 176.129]",
    "dlc_cat_(176.129, 200.9]",
    "dlc_cat_(200.9, inf]",
    "dltt_cat_(39.38, 327.85]",
    "dltt_cat_(327.85, 876.617]",
    "dltt_cat_(876.617, inf]",
    "capex_cat_(7.447, 79.55]",
    "capex_cat_(79.55, 5451.0]",
    "capex_cat_(5451.0, inf]",
    "revenue_cat_(0.174, 1248.817]",
    "revenue_cat_(1248.817, 4233.587]",
    "revenue_cat_(4233.587, inf]",
    "cce_cat_(5.619, 63.321]",
    "cce_cat_(63.321, inf]",
    "adv_cat_(0.3, 874.5]",
    "adv_cat_(874.5, inf]",
    "diff_positive",
    "roa_clip",
    "lev_sqrt",
    "intan_pow2",
    "rd_sqrt",
    "ppe_clip",
    "cash_holdings_sqrt",
    "adv_expenditure_positive",
    "diff_dta",
    "cfc_dta",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]
forward_elimination.remove("ta_log")
forward_elimination.remove("ppent_sqrt")
forward_elimination.remove("intant_sqrt")
forward_elimination.remove("roa")
forward_elimination.remove("lev")
forward_elimination.remove("intan")
forward_elimination.remove("rd_sqrt")
forward_elimination.remove("ppe")
forward_elimination.remove("cash_holdings_sqrt")

In [142]:
candidates_withoud_discr = [i for i in forward_elimination if "]" not in i]

In [143]:
grid_CV.best_params_

{'metric': 'minkowski', 'n_neighbors': 100, 'p': 1, 'weights': 'uniform'}

In [144]:
model = KNeighborsRegressor(**grid_CV.best_params_)

sf = SFS(
    model,
    k_features=(5, 15),
    forward=True,
    floating=False,
    verbose=0,
    scoring=mse,
    cv=5,
    n_jobs=-1,
)

sffit = sf.fit(
    df.loc[:, candidates_withoud_discr].values, df.loc[:, "etr"].values.ravel()
)

sf_features = df.loc[:, candidates_withoud_discr].columns[list(sffit.k_feature_idx_)]

sf_features

Index(['txt', 'xrd', 'ni', 'WB_GDPpc', 'ni_profit_20000', 'diff_dta',
       'etr_y_past', 'etr_y_ma'],
      dtype='object')

In [145]:
model = KNeighborsRegressor(**grid_CV.best_params_)

sf = SFS(
    model,
    k_features=(5, 15),
    forward=True,
    floating=False,
    verbose=0,
    scoring=mse,
    cv=5,
    n_jobs=-1,
)

sffit = sf.fit(df.loc[:, forward_elimination].values, df.loc[:, "etr"].values.ravel())

sf_features2 = df.loc[:, forward_elimination].columns[list(sffit.k_feature_idx_)]

sf_features2

Index(['txt', 'xrd', 'WB_GDPpc', 'txt_cat_(-63.011, -34.811]',
       'txt_cat_(24.415, 25.05]', 'txt_cat_(308.55, 327.531]',
       'pi_cat_(8108.5, inf]', 'dlc_cat_(176.129, 200.9]',
       'capex_cat_(5451.0, inf]', 'diff_dta', 'etr_y_past', 'etr_y_ma',
       'diff_ma'],
      dtype='object')

In [146]:
sf_one_more = [
    "rok",
    "diff",
    "roa",
    "lev",
    "intan",
    "rd",
    "ppe",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
]

In [147]:
sf = SFS(
    model,
    k_features=(5, 10),
    forward=True,
    floating=False,
    verbose=0,
    scoring=mse,
    cv=5,
    n_jobs=-1,
)

sffit = sf.fit(df.loc[:, sf_one_more].values, df.loc[:, "etr"].values.ravel())

sf_features3 = df.loc[:, sf_one_more].columns[list(sffit.k_feature_idx_)]

sf_features3

Index(['diff', 'adv_expenditure', 'capex2', 'cfc', 'dta'], dtype='object')

#### Relief

In [148]:
relief = [
    "rok",
    "ta",
    "txt",
    "pi",
    "str",
    "xrd",
    "ni",
    "ppent",
    "intant",
    "dlc",
    "dltt",
    "capex",
    "revenue",
    "cce",
    "adv",
    "diff",
    "rd",
    "sale",
    "cash_holdings",
    "adv_expenditure",
    "capex2",
    "cfc",
    "dta",
    "capex2_scaled",
    "y_v2x_polyarchy",
    "y_e_p_polity",
    "y_BR_Democracy",
    "WB_GDPgrowth",
    "WB_GDPpc",
    "WB_Inflation",
    "rr_per_country",
    "rr_per_sector",
    "sektor_consumer discretionary",
    "sektor_consumer staples",
    "sektor_energy",
    "sektor_health care",
    "sektor_industrials",
    "sektor_materials",
    "sektor_real estate",
    "sektor_technology",
    "sektor_utilities",
    "gielda_2",
    "gielda_3",
    "gielda_4",
    "gielda_5",
    "txt_cat_(-63.011, -34.811]",
    "txt_cat_(-34.811, 0.488]",
    "txt_cat_(0.488, 24.415]",
    "txt_cat_(24.415, 25.05]",
    "txt_cat_(25.05, 308.55]",
    "txt_cat_(308.55, 327.531]",
    "txt_cat_(327.531, inf]",
    "pi_cat_(-8975.0, -1.523]",
    "pi_cat_(-1.523, 157.119]",
    "pi_cat_(157.119, 465.9]",
    "pi_cat_(465.9, 7875.5]",
    "pi_cat_(7875.5, 8108.5]",
    "pi_cat_(8108.5, inf]",
    "str_cat_(0.0875, 0.192]",
    "str_cat_(0.192, 0.28]",
    "str_cat_(0.28, inf]",
    "xrd_exists",
    "ni_profit",
    "ni_profit_20000",
    "dlc_cat_(42.262, 176.129]",
    "dlc_cat_(176.129, 200.9]",
    "dlc_cat_(200.9, inf]",
    "dltt_cat_(39.38, 327.85]",
    "dltt_cat_(327.85, 876.617]",
    "dltt_cat_(876.617, inf]",
    "capex_cat_(7.447, 79.55]",
    "capex_cat_(79.55, 5451.0]",
    "capex_cat_(5451.0, inf]",
    "revenue_cat_(0.174, 1248.817]",
    "revenue_cat_(1248.817, 4233.587]",
    "revenue_cat_(4233.587, inf]",
    "cce_cat_(5.619, 63.321]",
    "cce_cat_(63.321, inf]",
    "adv_cat_(0.3, 874.5]",
    "adv_cat_(874.5, inf]",
    "diff_positive",
    "roa_clip",
    "lev_sqrt",
    "intan_pow2",
    "ppe_clip",
    "adv_expenditure_positive",
    "diff_dta",
    "cfc_dta",
    "etr_y_past",
    "etr_y_ma",
    "diff_ma",
    "roa_ma",
    "lev_ma",
    "intan_ma",
    "ppe_ma",
    "sale_ma",
    "cash_holdings_ma",
    "roa_past",
    "lev_past",
    "intan_past",
    "ppe_past",
    "sale_past",
    "cash_holdings_past",
]

In [149]:
fs = ReliefF(n_neighbors=30, n_features_to_keep=5)

In [150]:
fs.fit(df.loc[:, relief].values, df.loc[:, "etr"].values.ravel())

In [151]:
relief1 = df.loc[:, relief].iloc[:, fs.top_features[0:10]].columns.tolist()

In [152]:
relief2 = df.loc[:, relief].iloc[:, fs.top_features[0:15]].columns.tolist()

In [153]:
relief3 = df.loc[:, relief].iloc[:, fs.top_features[0:20]].columns.tolist()

### Hyperparametes Tunning for each group of variables

In [154]:
param = {
    "n_neighbors": [5, 7, 10, 12, 15, 25, 40, 50],
    "weights": ["uniform", "distance"],
    "metric": ["minkowski", "manhattan", "chebyshev"],
    "p": [1, 2],
}
mse = make_scorer(mean_squared_error, greater_is_better=True)

In [155]:
def cv_proc(var):
    model = KNeighborsRegressor()
    grid_CV = GridSearchCV(
        model, param, cv=5, scoring=mse, return_train_score=True, n_jobs=-1
    )
    grid_CV.fit(df.loc[:, var].values, df.loc[:, "etr"].values.ravel())
    print(grid_CV.best_params_)
    print(grid_CV.best_score_)

In [156]:
cv_proc(benchmark)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}
0.025545937441672474


In [157]:
cv_proc(benchmark2)

{'metric': 'minkowski', 'n_neighbors': 5, 'p': 1, 'weights': 'uniform'}
0.023609733866691088


In [158]:
cv_proc(mi_features_25)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.02370053011707496


In [159]:
cv_proc(mi_features_35)

{'metric': 'minkowski', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.0240364305155829


In [160]:
cv_proc(mi_features_50)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.02384039090887278


In [161]:
cv_proc(br_features)

{'metric': 'minkowski', 'n_neighbors': 5, 'p': 2, 'weights': 'distance'}
0.022355358767371106


In [162]:
cv_proc(mi_features)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.02346780295141025


In [163]:
cv_proc(corr_features)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.0239902552027932


In [164]:
cv_proc(sf_features)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.02199529688408505


In [165]:
cv_proc(sf_features2)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.02215778567896813


In [166]:
cv_proc(sf_features3)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.026571826392578657


In [167]:
cv_proc(relief1)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.02814496766754556


In [168]:
cv_proc(relief2)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.027816012510930698


In [169]:
cv_proc(relief3)

{'metric': 'chebyshev', 'n_neighbors': 5, 'p': 1, 'weights': 'distance'}
0.02848616264277109


### Final models comparison - winner obtaining

We would like to fight against data leakage in our CV, so we will treat it like a panel problem with a rolling window. We now based on our experience that this kind of approach is crucial to fight against overfitting.

Sliding window:
 * T: 2005 - 2008; V: 2009
 * T: 2005 - 2009; V: 2010
 * T: 2005 - 2010; V: 2011
 * ...

In [170]:
df = df.sort_values(by="rok").reset_index(drop=True)

In [171]:
def proper_CV(x, y, model, display_res=False):
    train_score = list()
    valid_score = list()
    train_indexes = [0, 1452]
    valid_indexes = [1452, 1815]
    for i in range(0, 6):
        train_x = x[x.index.isin(range(train_indexes[0], train_indexes[1]))]
        train_y = y[y.index.isin(range(train_indexes[0], train_indexes[1]))]
        valid_x = x[x.index.isin(range(valid_indexes[0], valid_indexes[1]))]
        valid_y = y[y.index.isin(range(valid_indexes[0], valid_indexes[1]))]

        model.fit(train_x.values, train_y.values.ravel())

        pred_y_train = model.predict(train_x.values)
        rmse = np.sqrt(mean_squared_error(train_y, pred_y_train))
        train_score.append(rmse)

        pred_y_val = model.predict(valid_x.values)
        rmse = np.sqrt(mean_squared_error(valid_y, pred_y_val))
        valid_score.append(rmse)

        train_indexes = [0, valid_indexes[1]]
        valid_indexes = [train_indexes[1], valid_indexes[1] + 363]

    if display_res == True:
        view = pd.DataFrame([train_score, valid_score]).T.rename(
            columns={0: "cv_train", 1: "cv_val"}
        )
        display(view)
        return train_score, valid_score, view
    else:
        return train_score, valid_score

In [172]:
hp = [
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "uniform"},
    {"metric": "minkowski", "n_neighbors": 5, "p": 1, "weights": "uniform"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "minkowski", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "minkowski", "n_neighbors": 5, "p": 2, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
    {"metric": "chebyshev", "n_neighbors": 5, "p": 1, "weights": "distance"},
]

In [174]:
model = KNeighborsRegressor(**hp[0])
var = benchmark
cv_output0 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.125798,0.152150
1,0.123955,0.145769
2,0.124362,0.156652
3,0.125836,0.151432
4,0.124683,0.135134
5,0.122922,0.144350


In [175]:
model = KNeighborsRegressor(**hp[1])

var = benchmark2
cv_output1 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.118046,0.139376
1,0.118018,0.142649
2,0.116764,0.156278
3,0.118558,0.154163
4,0.118302,0.135649
5,0.116839,0.138028


In [190]:
model = KNeighborsRegressor(**hp[2])
var = br_features
cv_output2 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.0,0.153015
1,0.0,0.135555
2,0.0,0.168149
3,0.0,0.155667
4,0.0,0.140242
5,0.0,0.150254


In [177]:
model = KNeighborsRegressor(**hp[3])
var = mi_features
cv_output3 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.149097
1,0.030226,0.145741
2,0.034400,0.161851
3,0.033165,0.157378
4,0.031101,0.129559
5,0.030100,0.139105


In [178]:
model = KNeighborsRegressor(**hp[4])
var = corr_features
cv_output4 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.0,0.158349
1,0.0,0.137706
2,0.0,0.157571
3,0.0,0.152803
4,0.0,0.137995
5,0.0,0.146224


In [179]:
model = KNeighborsRegressor(**hp[5])
var = sf_features
cv_output5 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.0,0.154890
1,0.0,0.136028
2,0.0,0.172723
3,0.0,0.156113
4,0.0,0.136444
5,0.0,0.147728


In [180]:
model = KNeighborsRegressor(**hp[6])
var = sf_features2
cv_output6 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.0,0.154905
1,0.0,0.131321
2,0.0,0.170680
3,0.0,0.157912
4,0.0,0.133249
5,0.0,0.150110


In [181]:
model = KNeighborsRegressor(**hp[7])
var = sf_features3
cv_output7 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.028794,0.150236
1,0.031273,0.144547
2,0.035171,0.165853
3,0.033852,0.154481
4,0.031230,0.152121
5,0.030156,0.153486


In [182]:
model = KNeighborsRegressor(**hp[8])
var = relief1
cv_output8 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.149543
1,0.030226,0.155197
2,0.033437,0.163921
3,0.033027,0.158555
4,0.030760,0.142748
5,0.030016,0.157629


In [183]:
model = KNeighborsRegressor(**hp[9])
var = relief2
cv_output9 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.158109
1,0.030226,0.148959
2,0.034045,0.159197
3,0.032434,0.156852
4,0.030994,0.142990
5,0.029527,0.148380


In [184]:
model = KNeighborsRegressor(**hp[10])
var = relief3
cv_output10 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.161813
1,0.030226,0.162003
2,0.034400,0.158788
3,0.033165,0.162929
4,0.031101,0.145962
5,0.030100,0.153895


In [185]:
model = KNeighborsRegressor(**hp[11])
var = mi_features_25
cv_output11 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.163037
1,0.030226,0.141467
2,0.034400,0.164054
3,0.033165,0.154349
4,0.031101,0.130668
5,0.030100,0.142010


In [186]:
model = KNeighborsRegressor(**hp[12])
var = mi_features_35
cv_output12 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.028174,0.158361
1,0.030226,0.145782
2,0.034400,0.156311
3,0.033165,0.151534
4,0.031101,0.138274
5,0.030100,0.143354


In [187]:
model = KNeighborsRegressor(**hp[13])
var = mi_features_50
cv_output13 = proper_CV(df.loc[:, var], df.loc[:, "etr"], model, display_res=True)

,cv_train,cv_val
0,0.0,0.144637
1,0.0,0.151558
2,0.0,0.161024
3,0.0,0.157175
4,0.0,0.136192
5,0.0,0.144896


In [191]:
pd.DataFrame(
    [
        cv_output0[2].mean().tolist(),
        cv_output1[2].mean().tolist(),
        cv_output2[2].mean().tolist(),
        cv_output3[2].mean().tolist(),
        cv_output4[2].mean().tolist(),
        cv_output5[2].mean().tolist(),
        cv_output6[2].mean().tolist(),
        cv_output7[2].mean().tolist(),
        cv_output8[2].mean().tolist(),
        cv_output9[2].mean().tolist(),
        cv_output10[2].mean().tolist(),
        cv_output11[2].mean().tolist(),
        cv_output12[2].mean().tolist(),
        cv_output13[2].mean().tolist(),
    ],
    columns=["train_mean", "test_mean"],
)

,train_mean,test_mean
0,0.124593,0.147581
1,0.117755,0.144357
2,0.000000,0.150480
3,0.031194,0.147122
4,0.000000,0.148441
5,0.000000,0.150654
6,0.000000,0.149696
7,0.031746,0.153454
8,0.030940,0.154599
9,0.030900,0.152414


In [192]:
pd.DataFrame(
    [
        cv_output0[2].std().tolist(),
        cv_output1[2].std().tolist(),
        cv_output2[2].std().tolist(),
        cv_output3[2].std().tolist(),
        cv_output4[2].std().tolist(),
        cv_output5[2].std().tolist(),
        cv_output6[2].std().tolist(),
        cv_output7[2].std().tolist(),
        cv_output8[2].std().tolist(),
        cv_output9[2].std().tolist(),
        cv_output10[2].std().tolist(),
        cv_output11[2].std().tolist(),
        cv_output12[2].std().tolist(),
        cv_output13[2].std().tolist(),
    ],
    columns=["train_std", "test_std"],
)

,train_std,test_std
0,0.001119,0.007571
1,0.000764,0.008739
2,0.000000,0.011603
3,0.002255,0.011838
4,0.000000,0.009272
5,0.000000,0.013850
6,0.000000,0.015121
7,0.002362,0.007018
8,0.001982,0.007458
9,0.002099,0.006560


Second model seems to be the best one! We see that our intuition was quite good - binary variables should be removed!

In [193]:
print(benchmark2)

['ta', 'txt', 'pi', 'str', 'xrd', 'ni', 'ppent', 'intant', 'dlc', 'dltt', 'capex', 'revenue', 'cce', 'adv', 'diff', 'roa', 'lev', 'intan', 'rd', 'ppe', 'sale', 'cash_holdings', 'adv_expenditure', 'capex2', 'cfc', 'dta', 'y_v2x_polyarchy', 'WB_GDPgrowth', 'WB_GDPpc', 'WB_Inflation', 'rr_per_country', 'rr_per_sector', 'etr_y_past', 'etr_y_ma', 'diff_ma', 'roa_ma', 'lev_ma', 'intan_ma', 'ppe_ma', 'sale_ma', 'cash_holdings_ma', 'roa_past', 'lev_past', 'intan_past', 'ppe_past', 'sale_past', 'cash_holdings_past']


### Fit final model and save it

In [194]:
model = KNeighborsRegressor(**hp[1])
model.fit(df.loc[:, benchmark2].values, df.loc[:, "etr"].values.ravel())

KNeighborsRegressor(p=1)

In [195]:
filename = "../models/knn.sav"

In [196]:
pickle.dump(model, open(filename, "wb"))